In [44]:
import pandas as pd
import numpy as np

# Neural Network Class

In [45]:
class NeuralNetwork:
    
    def __init__(self,inputNodes,hiddenList,hiddenActivations,outputNodes=1,outputActivation="SIGMOID"):
        self.inputNodes=inputNodes
        self.outputNodes=outputNodes
        self.hiddenList=[inputNodes]+hiddenList+[outputNodes]
        # Nodes in each Layer (Also stores corresponding to input and output layer)
        self.Activations=["None"]+hiddenActivations+[outputActivation]
        # Activation Functions (Also stores corresponding to input and output layer)
        self.weight=[]
        self.os=[]
        self.net=[]
        self.deltas=[]
        self.loss=0
        self.trainsteps=0
        self.epoch=0
    
    # Initialise weight Matrices Randomly
    def initialiseWeight(self):
        layers=len(self.hiddenList)
        self.trainsteps=0
        self.epoch=0
        
        for i in range(1,layers,1):
            self.weight.append(np.random.randn(self.hiddenList[i],self.hiddenList[i-1]+1))
    
    # Applies Activation Function
    def applyActivation(self,vector,activation="None"):
        if(activation=="RELU"):
            return vector*(vector>=0)
        elif(activation=="SIGMOID"):
            return (1.0/(1+np.exp(-vector)))
        else:
            return 1*vector
    
    def append1(self,x):
        [instances,attr]=x.shape
        bias=np.ones([instances,1])
        return np.concatenate((bias,x),axis=1)
    
    # Forward Pass a batch through the network
    def forwardPass(self,batchX):
        
        layers=len(self.weight)
        self.os=[self.append1(batchX)]
        self.net=[self.append1(batchX)]
                
        for i in range(layers):
            x=self.os[i]
            netj=np.matmul(x,self.weight[i].T)
            o=(self.applyActivation(netj,self.Activations[i+1]))
            if(i==layers-1):
                self.net.append(netj)
                self.os.append(o)
            else:
                self.net.append(self.append1(netj))
                self.os.append(self.append1(o))
    
    # Euclidean Loss -> if Changed, gradient J wrt o must be updated
    def lossfunction(self,pred,y):
        return (pred-y)**2
    
    # Function returning loss on batch
    def getLoss(self,batchX,batchY):
        self.forwardPass(batchX)
        layers=len(self.os)
        finalOutput=self.os[layers-1]
        return np.sum(self.lossfunction(finalOutput,batchY))

    # Gradient of o wrt net 
    def gradient_o_net(self,o,net,activation="None"):
        if(activation=="RELU"):
            return (net>=0)
        elif(activation=="SIGMOID"):
            return o*(1-o)
        else:
            return (o==o)
    
    def gradient_o_net_layer(self,layerNo):
        return self.gradient_o_net(self.os[layerNo],self.net[layerNo],self.Activations[layerNo])
    
    # Gradient of J wrt o
    def gradient_J_o(self,o,y):
        return 2*(o-y)        

    # Backward Pass a batch through the network updating Weight Matrices   
    def backwardPass(self,batchX,batchY,learningRate=0.01):
                
        layers=len(self.os)
#         m=float(1)
        m=float((batchX.shape)[0])
        self.deltas=[]
        
        finalDelta=self.gradient_J_o(self.os[layers-1],batchY)*self.gradient_o_net_layer(layers-1)        
        self.deltas.append(finalDelta)
        weightMatrixLen=len(self.weight)
        
        for i in range(layers-2,0,-1):
            weightMatrixLen-=1
            current_o_net=self.gradient_o_net_layer(i)
            current_delta=np.matmul(self.deltas[0],self.weight[weightMatrixLen])
            current_delta=current_delta*current_o_net
            [_,attributes]=current_delta.shape
            self.deltas=[current_delta[:,1:attributes]]+self.deltas
        
        layers=len(self.weight)
         
        for i in range(layers-1,-1,-1):
            del_w=np.matmul(self.deltas[i].T,self.os[i])
            self.weight[i]-=(learningRate/m)*del_w

    # training the NN
    def train(self,X,Y,learningRate=0.01,batchMode=False,batchSize=100,epsilon=0.0001,epochsToRun=500):
        
        [instances,attributes]=X.shape
        [_,outputs]=Y.shape
        
        if(batchMode==False):
            batchSize=instances
        
        Trained=False
        prevLoss=self.getLoss(X,Y)
        epochs=0
        
        while(not Trained and epochs<epochsToRun):
            self.epoch+=1
            epochs+=1
            
            cur=0
            while(cur<instances):
                uplim=min(cur+batchSize,instances)
                batchX=X[cur:uplim,0:attributes]
                batchY=Y[cur:uplim,0:outputs]
                cur=uplim
                self.forwardPass(batchX)
                self.backwardPass(batchX,batchY,learningRate)
                self.trainsteps+=1
            
            curLoss=self.getLoss(X,Y)
            
            if(abs(curLoss-prevLoss)<epsilon):
                Trained=True
            
            prevLoss=curLoss
            if(self.epoch%20==0):
                print ("Epoch:",self.epoch,"Loss:",prevLoss,"Accuracy:",self.getAccuracy(X,Y))
        
    def predict(self,X):
        self.forwardPass(X)
        layers=len(self.os)
        return self.os[layers-1]
    
    def getAccuracy(self,X,Y):
        pred=self.predict(X)
        instances=float(X.shape[0])
        
        pred=(pred>=0.5).astype(int)
        
        return (np.sum(pred==Y))/instances
        
            
            
        

# Loading Data

In [46]:
xtrain=pd.read_csv("Dataset/NN/toy_data/toy_trainX.csv",header=None,sep=',').values
ytrain=pd.read_csv("Dataset/NN/toy_data/toy_trainY.csv",header=None,sep=',').values
xtest=pd.read_csv("Dataset/NN/toy_data/toy_testX.csv",header=None,sep=',').values
ytest=pd.read_csv("Dataset/NN/toy_data/toy_testY.csv",header=None,sep=',').values

In [47]:
# xtrain=pd.read_csv("Dataset/NN/mnist_data/MNIST_train.csv",header=None,sep=',').values
# ytrain=xtrain[:,784:785]
# xtrain=xtrain[:,0:784]
# xtest=pd.read_csv("Dataset/NN/mnist_data/MNIST_test.csv",header=None,sep=',').values
# ytest=xtest[:,784:785]
# xtest=xtest[:,0:784]
# ytrain=(ytrain==6).astype(int)
# ytest=(ytest==6).astype(int)
# xtrain=xtrain/255.0
# xtest=xtest/255.0

In [48]:
# print(xtest)

In [49]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(380, 2)
(380, 1)
(120, 2)
(120, 1)


# Testing NN

In [50]:
# NN_obj=NeuralNetwork(xtrain.shape[1],[5,5],["SIGMOID","SIGMOID"],ytrain.shape[1])
# NN_obj=NeuralNetwork(xtrain.shape[1],[5,5],["RELU","RELU"],ytrain.shape[1])
# NN_obj=NeuralNetwork(xtrain.shape[1],[5],["SIGMOID"],ytrain.shape[1])
NN_obj=NeuralNetwork(xtrain.shape[1],[5],["SIGMOID"],ytrain.shape[1])

In [51]:
NN_obj.initialiseWeight()

In [52]:
for weights in NN_obj.weight:
    print(weights.shape)

(5, 3)
(1, 6)


In [53]:
NN_obj.getLoss(xtrain,ytrain)

114.14875075963997

In [68]:
NN_obj.train(X=xtrain,Y=ytrain,learningRate=1,batchMode=True,batchSize=100,epochsToRun=20000)

Epoch: 51160 Loss: 39.42597430086405 Accuracy: 0.8789473684210526
Epoch: 51180 Loss: 39.238898155756054 Accuracy: 0.8789473684210526
Epoch: 51200 Loss: 39.059368729494324 Accuracy: 0.8789473684210526
Epoch: 51220 Loss: 38.88764585554378 Accuracy: 0.8789473684210526
Epoch: 51240 Loss: 38.723326047737785 Accuracy: 0.881578947368421
Epoch: 51260 Loss: 38.56600392234988 Accuracy: 0.881578947368421
Epoch: 51280 Loss: 38.41530699895039 Accuracy: 0.881578947368421
Epoch: 51300 Loss: 38.27089253547222 Accuracy: 0.881578947368421
Epoch: 51320 Loss: 38.13244236069398 Accuracy: 0.881578947368421
Epoch: 51340 Loss: 37.99965870062045 Accuracy: 0.8842105263157894
Epoch: 51360 Loss: 37.87226127606496 Accuracy: 0.8842105263157894
Epoch: 51380 Loss: 37.74998552298939 Accuracy: 0.8842105263157894
Epoch: 51400 Loss: 37.63258167453108 Accuracy: 0.8842105263157894
Epoch: 51420 Loss: 37.51981440455655 Accuracy: 0.8842105263157894
Epoch: 51440 Loss: 37.41146272065017 Accuracy: 0.8842105263157894
Epoch: 51460

Epoch: 53640 Loss: 33.65038275389479 Accuracy: 0.8947368421052632
Epoch: 53660 Loss: 33.63583392824772 Accuracy: 0.8947368421052632
Epoch: 53680 Loss: 33.6214071624412 Accuracy: 0.8973684210526316
Epoch: 53700 Loss: 33.60710153649816 Accuracy: 0.8973684210526316
Epoch: 53720 Loss: 33.59291611495622 Accuracy: 0.8973684210526316
Epoch: 53740 Loss: 33.578849945068185 Accuracy: 0.8973684210526316
Epoch: 53760 Loss: 33.56490205549797 Accuracy: 0.8973684210526316
Epoch: 53780 Loss: 33.55107145548518 Accuracy: 0.8973684210526316
Epoch: 53800 Loss: 33.537357134446836 Accuracy: 0.8973684210526316
Epoch: 53820 Loss: 33.523758061981205 Accuracy: 0.8973684210526316
Epoch: 53840 Loss: 33.51027318823557 Accuracy: 0.8973684210526316
Epoch: 53860 Loss: 33.496901444597846 Accuracy: 0.8973684210526316
Epoch: 53880 Loss: 33.48364174467075 Accuracy: 0.8973684210526316
Epoch: 53900 Loss: 33.47049298548698 Accuracy: 0.8973684210526316
Epoch: 53920 Loss: 33.45745404892418 Accuracy: 0.8973684210526316
Epoch: 

Epoch: 56260 Loss: 32.43578899238955 Accuracy: 0.9026315789473685
Epoch: 56280 Loss: 32.42989618713833 Accuracy: 0.9026315789473685
Epoch: 56300 Loss: 32.42403564552966 Accuracy: 0.9026315789473685
Epoch: 56320 Loss: 32.41820711031951 Accuracy: 0.9026315789473685
Epoch: 56340 Loss: 32.41241032714913 Accuracy: 0.9026315789473685
Epoch: 56360 Loss: 32.406645044500294 Accuracy: 0.9026315789473685
Epoch: 56380 Loss: 32.400911013651324 Accuracy: 0.9026315789473685
Epoch: 56400 Loss: 32.3952079886341 Accuracy: 0.9026315789473685
Epoch: 56420 Loss: 32.38953572619185 Accuracy: 0.9026315789473685
Epoch: 56440 Loss: 32.38389398573798 Accuracy: 0.9026315789473685
Epoch: 56460 Loss: 32.37828252931553 Accuracy: 0.9026315789473685
Epoch: 56480 Loss: 32.37270112155763 Accuracy: 0.9026315789473685
Epoch: 56500 Loss: 32.36714952964863 Accuracy: 0.9026315789473685
Epoch: 56520 Loss: 32.361627523286046 Accuracy: 0.9026315789473685
Epoch: 56540 Loss: 32.3561348746433 Accuracy: 0.9026315789473685
Epoch: 56

Epoch: 58880 Loss: 31.868800554005503 Accuracy: 0.9026315789473685
Epoch: 58900 Loss: 31.865653671490648 Accuracy: 0.9026315789473685
Epoch: 58920 Loss: 31.86252002654011 Accuracy: 0.9026315789473685
Epoch: 58940 Loss: 31.8593995407733 Accuracy: 0.9026315789473685
Epoch: 58960 Loss: 31.856292136387935 Accuracy: 0.9026315789473685
Epoch: 58980 Loss: 31.85319773615416 Accuracy: 0.9026315789473685
Epoch: 59000 Loss: 31.8501162634086 Accuracy: 0.9026315789473685
Epoch: 59020 Loss: 31.84704764204858 Accuracy: 0.9026315789473685
Epoch: 59040 Loss: 31.84399179652638 Accuracy: 0.9026315789473685
Epoch: 59060 Loss: 31.840948651843515 Accuracy: 0.9026315789473685
Epoch: 59080 Loss: 31.837918133545138 Accuracy: 0.9026315789473685
Epoch: 59100 Loss: 31.834900167714483 Accuracy: 0.9026315789473685
Epoch: 59120 Loss: 31.831894680967306 Accuracy: 0.9026315789473685
Epoch: 59140 Loss: 31.828901600446546 Accuracy: 0.9026315789473685
Epoch: 59160 Loss: 31.825920853816832 Accuracy: 0.9026315789473685
Epo

In [69]:
print (NN_obj.getAccuracy(xtrain,ytrain))
print (NN_obj.getAccuracy(xtest,ytest))

0.9
0.85
